In [1]:
#%pip install selenium
#%pip install bs4
#%pip install keyboard
#%pip install undetected-chromedriver
#%pip install setuptools
#%pip install tk

In [2]:
from selenium import webdriver
import keyboard
from bs4 import BeautifulSoup
import time
import undetected_chromedriver as uc
import tkinter as tk
from tkinter import ttk


In [56]:
COIN_URL = "https://dexscreener.com/solana/7sjs2dzcrwk2ytcznextdttbx2oyc6y4jmynwpt9tmqr"

In [58]:
def getText(driver):
    time.sleep(1)
    src = driver.page_source
    soup = BeautifulSoup(src, "html.parser")
    price = soup.find("div","custom-1baulvz")
    if price:
        return float(price.text.strip()[1:])
    return 0

In [59]:
def calcValue(price,numberOfCoins):
    return price * numberOfCoins

In [60]:
def calcProfit(currentPrice,entryPrice,numberOfCoins):
    return (currentPrice - entryPrice) * numberOfCoins

In [61]:
def update_value(driver,numberOfCoinsVar,entryPriceVar, entryValueLabel, currentValueLabel, profitLabel):
    try:
        #update total value
        price = getText(driver)
        number_of_coins = float(numberOfCoinsVar.get())
        value = calcValue(price, number_of_coins)
        currentValueLabel.config(text=f"Current Value: {value:.2f}")

        # If entry price is provided, calculate profit
        entry_price = float(entryPriceVar.get())
        entryValue = calcValue(entry_price, number_of_coins)
        entryValueLabel.config(text=f"Entry Value: {entryValue:.2f}")

        if entry_price:
            entry_price = float(entry_price)
            profit = calcProfit(price, entry_price, number_of_coins)
            profitLabel.config(text=f"Total profit: {profit:.2f}")
            if profit > 0:
                profitLabel.config(foreground="green")
            elif profit < 0:
                profitLabel.config(foreground="red")
            else:
                profitLabel.config(foreground="black")
        else:
            profitLabel.config(text="Total profit: N/A", foreground="black")
        
        currentValueLabel.after(1000, update_value, driver,numberOfCoinsVar,entryPriceVar, entryValueLabel, currentValueLabel, profitLabel)
    except Exception as e:
        print(f"Error: {e}")

In [62]:
def main(numberOfCoinsVar, entryPriceVar, entryValueLabel, currentValueLabel, profitLabel):
    driver = uc.Chrome()
    driver.get(COIN_URL)
    update_value(driver,numberOfCoinsVar,entryPriceVar, entryValueLabel, currentValueLabel, profitLabel)
    

In [63]:
def create_gui():
    root = tk.Tk()
    root.title("Crypto Value Tracker")
    
    # Input for number of coins
    ttk.Label(root, text="Number of Coins:").grid(row=0, column=0, padx=10, pady=10)
    numberOfCoinsVar = tk.StringVar()
    ttk.Entry(root, textvariable=numberOfCoinsVar).grid(row=0, column=1, padx=10, pady=10)

    # Input for entry price
    ttk.Label(root, text="Entry Price:").grid(row=1, column=0, padx=10, pady=10)
    entryPriceVar = tk.StringVar()
    ttk.Entry(root, textvariable=entryPriceVar).grid(row=1, column=1, padx=10, pady=10)

    #Label to display the entry value
    entryValueLabel = ttk.Label(root, text="Entry value: ", font=("Helvetica", 14))
    entryValueLabel.grid(row=2, column=0, columnspan=2, pady=10)
    
    # Label to display the current value
    currentValueLabel = ttk.Label(root, text="Current value: ", font=("Helvetica", 14))
    currentValueLabel.grid(row=3, column=0, columnspan=2, pady=10)

    # Label to display the profit
    profitLabel = ttk.Label(root, text="Total profit: N/A", font=("Helvetica", 14))
    profitLabel.grid(row=4, column=0, columnspan=2, pady=10)

     # Button to start tracking
    ttk.Button(root, text="Start Tracking", 
               command=lambda: main(numberOfCoinsVar, entryPriceVar, entryValueLabel, currentValueLabel, profitLabel)).grid(row=5, column=0, columnspan=2, pady=10)
    
    root.mainloop()

In [55]:
if __name__ == "__main__":
    create_gui()